In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.read_csv('data\dev_wings_agg_span_2024_01_01.csv')
df = raw_df[['shape', 'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive', 'cust_essential', 'cust_medicalcert',
             'cust_urgent', 'cust_lifesupport', 'cust_total', 'downstream_cust_industrial', 'downstream_cust_commercial', 'downstream_cust_residential',
             'downstream_cust_sensitive', 'downstream_cust_essential', 'downstream_cust_medicalcert', 'downstream_cust_urgent', 'downstream_cust_lifesupport',
             'downstream_cust_total']]

In [ ]:
df

In [ ]:
import folium
from shapely import wkt
from pyproj import Transformer

transformer = Transformer.from_crs("EPSG:2230", "EPSG:4326", always_xy=True)

first_geom = wkt.loads(df['shape'].iloc[0])
first_coord = list(first_geom.coords)[0]
first_lon, first_lat = transformer.transform(*first_coord)
m = folium.Map(location=(first_lat, first_lon), zoom_start=14)

for idx, row in df.head(10000).iterrows():
    # Convert WKT to geometry
    geom = wkt.loads(row['shape'])
    coords = list(geom.coords)
    # Transform coordinates from source CRS to lat/lon (EPSG:4326)
    transformed = [transformer.transform(x, y) for x, y in coords]
    # folium expects (lat, lon) so swap the order
    folium_coords = [(lat, lon) for lon, lat in transformed]
    # Add the line to the map
    folium.PolyLine(locations=folium_coords, color="blue", weight=5).add_to(m)

m

In [ ]:
polygons = pd.read_csv('data\src_vri_snapshot_2024_03_20.csv')

polygons

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely import wkt
from shapely.ops import transform
from pyproj import Transformer

gdf_lines = gpd.GeoDataFrame(
    df,
    geometry=gpd.GeoSeries.from_wkt(df['shape'])
)
gdf_lines.set_crs("EPSG:2230", inplace=True)

gdf_polygons = gpd.GeoDataFrame(
    polygons,
    geometry=gpd.GeoSeries.from_wkt(polygons['shape'])
)
gdf_polygons.set_crs("EPSG:4326", inplace=True)

def remove_z(x, y, z=None):
    return (x, y)

gdf_polygons['geometry'] = gdf_polygons.geometry.apply(lambda geom: transform(remove_z, geom))

# ---------------------------
# 3. Transform Lines to EPSG:4326
# ---------------------------
lines_transformer = Transformer.from_crs("EPSG:2230", "EPSG:4326", always_xy=True)
gdf_lines = gdf_lines.to_crs("EPSG:4326")

# ---------------------------
# 4. Spatial Join & Summing cust_total
# ---------------------------
gdf_lines_subset = gdf_lines

joined = gpd.sjoin(gdf_lines_subset, gdf_polygons, how="inner", predicate="intersects")

# Define the list of customer-related columns to sum.
cols_to_sum = [
    'cust_industrial', 'cust_commercial', 'cust_residential', 'cust_sensitive',
    'cust_essential', 'cust_medicalcert', 'cust_urgent', 'cust_lifesupport',
    'cust_total', 'downstream_cust_industrial', 'downstream_cust_commercial',
    'downstream_cust_residential', 'downstream_cust_sensitive', 'downstream_cust_essential',
    'downstream_cust_medicalcert', 'downstream_cust_urgent', 'downstream_cust_lifesupport',
    'downstream_cust_total'
]

cust_sum = joined.groupby("index_right")[cols_to_sum].sum()

gdf_polygons = gdf_polygons.join(cust_sum, how='left')

gdf_polygons[cols_to_sum] = gdf_polygons[cols_to_sum].fillna(0)

gdf_polygons[cols_to_sum]

In [ ]:
gdf_polygons.columns

In [ ]:
df = gdf_polygons

import pandas as pd

weights = {
    'lifesupport': 8,
    'urgent': 7,
    'medicalcert': 6,
    'essential': 5,
    'sensitive': 4,
    'residential': 3,
    'industrial': 2,
    'commercial': 1
}

# Calculate cust_risk_score
df['cust_risk_score'] = (
    weights['lifesupport'] * df['cust_lifesupport'] +
    weights['urgent'] * df['cust_urgent'] +
    weights['medicalcert'] * df['cust_medicalcert'] +
    weights['essential'] * df['cust_essential'] +
    weights['sensitive'] * df['cust_sensitive'] +
    weights['residential'] * df['cust_residential'] +
    weights['industrial'] * df['cust_industrial'] +
    weights['commercial'] * df['cust_commercial']
)

# Calculate downstream_cust_risk_score
df['downstream_cust_risk_score'] = (
    weights['lifesupport'] * df['downstream_cust_lifesupport'] +
    weights['urgent'] * df['downstream_cust_urgent'] +
    weights['medicalcert'] * df['downstream_cust_medicalcert'] +
    weights['essential'] * df['downstream_cust_essential'] +
    weights['sensitive'] * df['downstream_cust_sensitive'] +
    weights['residential'] * df['downstream_cust_residential'] +
    weights['industrial'] * df['downstream_cust_industrial'] +
    weights['commercial'] * df['downstream_cust_commercial']
)

consolidated_df = df[['shape', 'cust_total', 'downstream_cust_total', 'cust_risk_score', 'downstream_cust_risk_score']]

In [ ]:
min_score = consolidated_df['cust_risk_score'].min()
max_score = consolidated_df['cust_risk_score'].max()

if max_score - min_score == 0:
    consolidated_df['cust_risk_score_scaled'] = 100
else:
    consolidated_df['cust_risk_score_scaled'] = ((consolidated_df['cust_risk_score'] - min_score) / (max_score - min_score)) * 100

min_downstream = consolidated_df['downstream_cust_risk_score'].min()
max_downstream = consolidated_df['downstream_cust_risk_score'].max()

if max_downstream - min_downstream == 0:
    consolidated_df['downstream_cust_risk_score_scaled'] = 100
else:
    consolidated_df['downstream_cust_risk_score_scaled'] = ((consolidated_df['downstream_cust_risk_score'] - min_downstream) / (max_downstream - min_downstream)) * 100

consolidated_df

In [ ]:
consolidated_df.to_csv('life_output.csv')

In [ ]:
center = [gdf_polygons.unary_union.centroid.y, gdf_polygons.unary_union.centroid.x]
m = folium.Map(location=center, zoom_start=12)


tooltip = folium.GeoJsonTooltip(
    fields=['cust_total', 'downstream_cust_total'],
    aliases=['Cust Total:', 'Downstream Cust Total:'],
    localize=True
)

# Add the polygons layer.
folium.GeoJson(
    gdf_polygons.__geo_interface__,
    name="Polygons",
    style_function=lambda feature: {
        'fillColor': 'lightblue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.3,
    },
    tooltip=tooltip
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
m.save("prototype.html")